In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tika import parser
import pickle
import os
import re
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
########################################################
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Install GPU version of TF")


########################################################

2.1.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14915679700107670387
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10068632535
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12334014567914866986
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Default GPU Device: /device:GPU:0


In [2]:
# inputs
input_folder = 'input/'
files = os.listdir(input_folder)

input_data = {}

for file in files:
    extention = file[file.find('.'):]
    if extention == '.pdf':
        content = parser.from_file(input_folder+file)['content']
        print(input_folder+file)    
    elif extention == '.txt':
        with open(input_folder+file, "r", encoding="utf8") as f:
            content = f.read()
        print(input_folder+file)  
        
    else:
        print("ERROR Unindetified type")
        break

    input_data[file[:file.find('.')]] = content
    

input_data.keys()


input/babits_összes.pdf
input/karinthy_összes.txt
input/petofi_összes.txt


dict_keys(['babits_összes', 'karinthy_összes', 'petofi_összes'])

In [3]:
print(input_data['karinthy_összes'][:500])


Karinthy Frigyes 
összes költeménye
Vérmező, 795. május
                 I
Ó, csudacsöndes májusi éj!
Alszik remegőn a hűs Duna tükre -
Lent fekszik a hold a víz fenekén.
Távol a Gellért. - Messze az éjben
Nyulnak a sávok, zöld-feketén.
A Várhegy fekete tömbje
Omlik az éjbe vakon -
Tompa rezes fény, lágy ütemekben
Visszaverődik az ablakokon.
Májusban az ifjú, nagy élet
Kéjtől zokogott, kéjétől a nyárnak -:
Éjfélben a hajóhíd alatt
Suhanni kezdtek az árnyak.

                 II
Éjfélben halk, h


In [4]:
# create general tokenization: combine -> filter -> tokenization

# combine texts

all_texts = ""

for k, v in input_data.items():
    print(k, "is added")
#     print(v)
    all_texts += v

with open('all_text.txt', 'w', encoding='utf-8') as w:
    w.write(all_texts)



babits_összes is added
karinthy_összes is added
petofi_összes is added


In [5]:
# filter

filter_char = [char for char in "()*+0123456789§«»–✝'[]#\/:…;"]
# filter_char.extend(["\n", "\t"])

replace_dict = {'á': ["â","ä"],
                "é":["è", "ë"],
                'í':['î'],
                'ő':['ô'],
                'c':['č'],
                '"' : ['”','„'],
                "'": ['‘','’'],
                ".": ["- - - - - - -- - - - - - -- - - - - - -"],
                "! ":['!'],
                "? ":['?'],
                ", ":[','],
                ". ":['.'],
                "...":['. . .'],
                "  ": ["\n", "\t", "     ", "    ", "   ", "   ", "   ", "   "],
                " ":["  ","  ","  "]}

def filter_text(raw_text, filter_char, replace):
    raw_text = re.sub(r'http(.*)', '', raw_text)
    for k, v in replace.items():
        for i in v:
            raw_text = raw_text.replace(i,k)
    for char in filter_char:
        raw_text = raw_text.replace(char, '')
    return raw_text

filtered = filter_text(all_texts, filter_char, replace_dict)


print("BEFORE FILTERING:", "".join(sorted(set(all_texts.lower()))))
print("\n")
print("AFTER FILTERING:\n", "".join(sorted(set(filtered.lower()))))
# print(raw_text[5480:5648])


with open('filtered_text.txt', 'w', encoding='utf-8') as w:
    w.write(filtered)


BEFORE FILTERING: 	
 !"#'()*+,-./0123456789:;?[]abcdefghijklmnopqrstuvwxyz§«»áâäèéëíîóôöúüčőű–‘’‚”„…✝


AFTER FILTERING:
  !",-.?abcdefghijklmnopqrstuvwxyzáéíóöúüőű‚


In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([filtered])

# saving
with open('models/tokenizer_magyar.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
tokenizer.texts_to_sequences(["First"])

[[26, 12, 10, 7, 4]]

In [8]:
tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(["First"]))

['f i r s t']

In [9]:
# max_id = len(tokenizer.word_index) # number of distinct characters
# dataset_size = tokenizer.document_count # total number of characters
#max_id#, dataset_size

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 52
n_steps = 100

# train_size = len(encoded) * 90 // 100
# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])


model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, batch_size],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(batch_size, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")







# TOKENIZE THE GIVEN INPUT SET
for poet, poems in input_data.items():
    filtered_input = filter_text(poems, filter_char, replace_dict)
    [encoded] = np.array(tokenizer.texts_to_sequences([filtered_input])) - 1
    train_size = len(encoded) * 90 // 100
    dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

    window_length = n_steps + 1 
    dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    dataset= dataset.shuffle(10000).batch(batch_size)
    dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

    dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=batch_size), Y_batch))
    dataset = dataset.prefetch(1)
    
    earlyStopping = EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(poet + '.mdl_wts.hdf5', save_best_only=True, monitor='loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
    history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=50, callbacks=[earlyStopping, mcp_save,reduce_lr_loss])
    model.save('models/' + poet + "_model.h5")
    
#     for X_batch, Y_batch in dataset.take(1):
#         print(X_batch.shape, Y_batch.shape)

Train for 10927 steps
Epoch 1/50
10927/10927 [==============================] - 149s 14ms/step - loss: 1.4968
Epoch 2/50
10927/10927 [==============================] - 144s 13ms/step - loss: 1.3675
Epoch 3/50
10927/10927 [==============================] - 143s 13ms/step - loss: 1.3750
Epoch 4/50
10927/10927 [==============================] - 142s 13ms/step - loss: 1.3925
Epoch 5/50
10927/10927 [==============================] - 144s 13ms/step - loss: 1.4113
Epoch 6/50
10927/10927 [==============================] - 142s 13ms/step - loss: 1.4277
Epoch 7/50
10927/10927 [==============================] - 142s 13ms/step - loss: 1.4451
Epoch 8/50
10927/10927 [==============================] - 141s 13ms/step - loss: 1.4620
Epoch 9/50
10922/10927 [============================>.] - ETA: 0s - loss: 1.4814
Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
10927/10927 [==============================] - 141s 13ms/step - loss: 1.4814
Epoch 10/50
10927/10927 [==========

In [10]:
# Encoding the full text (substract 1 to get IDs from 0-38 rather than 1-39)
# [encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) - 1
# train_size = dataset_size * 90 // 100
# train_size = len(encoded) * 90 // 100
# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])


# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# [encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) -1

# n_steps = 100
# window_length = n_steps + 1 
# dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
# # dataset = dataset.window(window_length, shift=1, drop_remainder=True)
# dataset = dataset.flat_map(lambda window: window.batch(window_length))

# np.random.seed(42)
# tf.random.set_seed(42)

# batch_size = 50
# dataset= dataset.shuffle(10000).batch(batch_size)
# dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

# dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
# dataset = dataset.prefetch(1)

In [11]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax')),
# ])



# history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)






# model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer="adam")


In [12]:
# history = model.fit(dataset, epochs = 20)

# %load_ext tensorboard
# %tensorboard --logdir=logs

In [13]:
# model_name = "Babits_2_GRU_1_TimeDist_Dense_no_dropout"
# !mkdir -p model_name
# model.save(model_name)

# new_model = tf.keras.models.load_model('saved_model/my_model')

In [14]:
# # USING THE MODEL TO PRED CHAR
# def preprocess(texts):
#     X = np.array(tokenizer.texts_to_sequences(texts)) - 1
#     return tf.one_hot(X, max_id)

# X_new = preprocess(["ej mi a k"])
# Y_pred = model.predict_classes(X_new)
# tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

In [15]:
# tf.random.set_seed(42)
# tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()


In [16]:
# def next_char(text, temperature=1):
#     X_new = preprocess([text])
#     y_proba = model.predict(X_new)[0, -1:, :]
#     rescaled_logits = tf.math.log(y_proba) / temperature
#     char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
#     return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [17]:
# tf.random.set_seed(42)
# next_char("valam", temperature=1)

In [18]:

# def complete_text(text, n_chars=50, temperature=1):
#     for _ in range(n_chars):
#         text += next_char(text, temperature)
#     return text

In [19]:
# tf.random.set_seed(42)

# # print(complete_text("t", temperature=0.2))
# print(complete_text("Magyar ",n_chars=250 ,temperature=0.8))
# # print(complete_text("t", temperature=2))